# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

### Содержание ###

- **Описание данных**
- **Задачи**
- **Шаг 1. Открыть файл с данными и изучить его.**
    - ***Вывод по первому шагу***
- **Шаг 2. Разделение данных на выборки.**
    - ***Вывод по второму шагу***
- **Шаг 3. Исследование разных моделей.**
    - ***Вывод по третьему шагу***
- **Шаг 4. Проверка качество модели на тестовой выборке.**
    - ***Вывод по четвертому шагу***
- **Шаг 5. Проверка модели на адекватность.**
    - ***Общий вывод***

### Описание данных ###

Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. Известно:

- сalls — количество звонков,
- minutes — суммарная длительность звонков в минутах,
- messages — количество sms-сообщений,
- mb_used — израсходованный интернет-трафик в Мб,
- is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

### Задачи ###

- Шаг 1. Открыть файл с данными и изучить его.
- Шаг 2. Разделение данных на обучающую, валидационную и тестовую выборки.
- Шаг 3. Исследование качества разных моделей.
- Шаг 4. Проверка качество модели на тестовой выборке.
- Шаг 5. Проверка модели на вменяемость.

## Откройте и изучите файл

In [1]:
# Изначально импортируем библиотеки 
import pandas as pd 
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
# Откроем файл и изучем его
df = pd.read_csv('/datasets/users_behavior.csv')
df.info()
df.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0
8,7.0,43.39,3.0,2538.67,1
9,90.0,665.41,38.0,17358.61,0


In [2]:
# Видно что столбцы calls и messages не в правильном формате, изменим их 
df['calls']=df['calls'].astype('int')
df['messages']=df['messages'].astype('int')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   int64  
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   int64  
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 125.7 KB


***Вывод: с данными все хорошо по условию(пропусков нет), поменяли только типы данных в 2-х столбцах***

## Разбейте данные на выборки

In [4]:
#Посмотрим как распределены классы в нашей выборке
df.is_ultra.value_counts(normalize=True)

0    0.693528
1    0.306472
Name: is_ultra, dtype: float64

*Видем что 70% составляет тариф "Смарт" и 30% тариф "Ультра"

In [6]:
#Определим наш random_state, который будем использовать для всех методов 
RANDOM_STATE = 125
#т.к. нам нужны три выборки, разобьем их по % на 60-20-20
df_train, df_valid_ant_test = train_test_split(df, test_size=0.4, random_state=RANDOM_STATE, stratify=df.is_ultra)
# теперь разделим тестовую выборку от валидационной
df_valid, df_test = train_test_split(df_valid_ant_test, test_size=0.5, random_state=RANDOM_STATE)
del df_valid_ant_test

In [7]:
print(f'Размер обучающей выборки составляет: {df_train.shape[0] / len(df):.1%}')
print(f'Размер тестовой выборки составляет: {df_test.shape[0] / len(df):.1%}')
print(f'Размер валидационой выборки составляет: {df_valid.shape[0] / len(df):.1%}')

Размер обучающей выборки составляет: 60.0%
Размер тестовой выборки составляет: 20.0%
Размер валидационой выборки составляет: 20.0%


**Вывод: Разделили на тестовую выборку и валидационую, переходим к выбору модели**

## Исследуйте модели

In [8]:
# Определим переменную для сохранения лучших параметров модели
best_accuracy_dict = dict()

# функция, чтобы меньше дублировать код в циклах
def fill_best_accuracy_dict(model, modelName, accuracy):
    best_accuracy_dict['model'] = model
    best_accuracy_dict['modelName'] = modelName
    best_accuracy_dict['accuracy'] = accuracy
    return best_accuracy_dict

# значения по умолчанию
best_accuracy_dict = fill_best_accuracy_dict('Undefined', 'Undefined', 0)

In [9]:
# Для начала определим наши признаки

# тренировочные
train_features = df_train.drop(['is_ultra'], axis=1)
train_target = df_train['is_ultra']
# валидационные
valid_features = df_valid.drop(['is_ultra'], axis=1)
valid_target = df_valid['is_ultra']
# тестовые
test_features = df_test.drop(['is_ultra'], axis=1)
test_target = df_test['is_ultra']

In [10]:
# исследуем дерево решений
for max_depth in range (2, 11, 1):
    modelDecisionTree = DecisionTreeClassifier(max_depth=max_depth, random_state=RANDOM_STATE)
    modelDecisionTree.fit(train_features, train_target)
    valid_predicted = modelDecisionTree.predict(valid_features)
    
    accuracy = accuracy_score(valid_target, valid_predicted)
    
    modelName = 'Descision Tree; max_depth= ' + str(max_depth)
    if (accuracy > best_accuracy_dict['accuracy']):
        best_accuracy_dict = fill_best_accuracy_dict(modelDecisionTree, modelName, accuracy)
    print(modelName,'; accuracy =', accuracy)

Descision Tree; max_depth= 2 ; accuracy = 0.7667185069984448
Descision Tree; max_depth= 3 ; accuracy = 0.776049766718507
Descision Tree; max_depth= 4 ; accuracy = 0.7822706065318819
Descision Tree; max_depth= 5 ; accuracy = 0.7838258164852255
Descision Tree; max_depth= 6 ; accuracy = 0.7713841368584758
Descision Tree; max_depth= 7 ; accuracy = 0.7807153965785381
Descision Tree; max_depth= 8 ; accuracy = 0.7869362363919129
Descision Tree; max_depth= 9 ; accuracy = 0.776049766718507
Descision Tree; max_depth= 10 ; accuracy = 0.7698289269051322


In [11]:
# исследуем случайный лес
for estim in range(5, 51, 5):
    for max_depth in range(5, 31, 5):
        modelRandomForest = RandomForestClassifier(n_estimators=estim, max_depth=max_depth, random_state=RANDOM_STATE)
        modelRandomForest.fit(train_features, train_target)
        valid_predicted = modelRandomForest.predict(valid_features)
        
        accuracy = accuracy_score(valid_target, valid_predicted)
        
        modelName = "Random Forest; n_estimators = " + str(estim) + " ; max_depth= " + str(max_depth)
        if (accuracy > best_accuracy_dict['accuracy']):
            best_accuracy_dict = fill_best_accuracy_dict(modelRandomForest, modelName, accuracy)
        print(modelName,'; accuracy =', accuracy)
    print() # для разделения вывода

Random Forest; n_estimators = 5 ; max_depth= 5 ; accuracy = 0.7916018662519441
Random Forest; n_estimators = 5 ; max_depth= 10 ; accuracy = 0.7884914463452566
Random Forest; n_estimators = 5 ; max_depth= 15 ; accuracy = 0.7791601866251944
Random Forest; n_estimators = 5 ; max_depth= 20 ; accuracy = 0.776049766718507
Random Forest; n_estimators = 5 ; max_depth= 25 ; accuracy = 0.7651632970451011
Random Forest; n_estimators = 5 ; max_depth= 30 ; accuracy = 0.749611197511664

Random Forest; n_estimators = 10 ; max_depth= 5 ; accuracy = 0.7947122861586314
Random Forest; n_estimators = 10 ; max_depth= 10 ; accuracy = 0.8009331259720062
Random Forest; n_estimators = 10 ; max_depth= 15 ; accuracy = 0.7744945567651633
Random Forest; n_estimators = 10 ; max_depth= 20 ; accuracy = 0.7838258164852255
Random Forest; n_estimators = 10 ; max_depth= 25 ; accuracy = 0.7791601866251944
Random Forest; n_estimators = 10 ; max_depth= 30 ; accuracy = 0.7791601866251944

Random Forest; n_estimators = 15 ; m

In [12]:
# уберем будущий ворнинг
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Ислледуем логистическую регрессию
modelLogisticRegression = LogisticRegression(random_state=RANDOM_STATE)
modelLogisticRegression.fit(train_features, train_target)
valid_predicted = modelLogisticRegression.predict(valid_features)

accuracy = accuracy_score(valid_target, valid_predicted)
        
modelName = 'Logistic Regression'
if (accuracy > best_accuracy_dict['accuracy']):
    best_accuracy_dict = fill_best_accuracy_dict(modelLogisticRegression, modelName, accuracy)
print(modelName,'; accuracy =', accuracy)

Logistic Regression ; accuracy = 0.7418351477449455


In [13]:
# Посмотрим, какая модель оказалась лучше всего
best_accuracy_dict

{'model': RandomForestClassifier(max_depth=10, n_estimators=45, random_state=125),
 'modelName': 'Random Forest; n_estimators = 45 ; max_depth= 10',
 'accuracy': 0.8040435458786936}

**Вывод:** 
*Лучший результат в точности показал случайный лес с параметрами n_estimators = 45 ; max_depth= 10*

*Худший результат в точночти показала логистическая регрессия*

*Довольно неплохо показало себя решающее дерево, при чем глубины 3 было достаточно*

*Из любопытного - независимо от числа деревьев, лучшие результаты показывал лес с глубиной 10*

## Проверьте модель на тестовой выборке

In [16]:
# проверим модель на тестовых данных
model = best_accuracy_dict['model']
test_predicted = model.predict(test_features)
print('Test accuracy =', accuracy_score(test_target, test_predicted))

Test accuracy = 0.8180404354587869


In [17]:
# из интереса проверим точность модели на исходных данных
train_predicted = model.predict(train_features)
print('Train accuracy =', accuracy_score(train_target, train_predicted))

Train accuracy = 0.8952282157676349


**Вывод : получили хорошую точность модели, при этом точность не вышла за 90% на исходных данных, фактор переобучения не критичен для нашей модели**

## (бонус) Проверьте модели на адекватность

In [18]:
# Проверим модель на адекватность
import random
random_predictions = pd.Series(test_target.apply(lambda x: round(random.random())), index=test_target.index)

# оценим точность на этих данных
print('Random predictions accuracy = ', accuracy_score(test_target, random_predictions))

Random predictions accuracy =  0.49144634525660963


**Общий вывод: Мы провели отличную работу и выявили модель.** 

Мы изучили наши данные и привели типы к нужным значениям

Создали три выборки по которым обучили и проверили нашу модель в пропорции 60% - 20% - 20%

Мы исследовали три различные модели с разными значениями гиперпараметров (Дерево решений, Случайный лес, Логистическую регрессию)

Мы определили лучшую модель и параметры для нее, измерив и сравнив точность наших предсказаний на валидационной выборке (в нашем случае лучший результат показал случайный лес с 25 деревьями и глубиной 10)

Мы проверили нашу модель на тестовых данных и они оказались очень близко к данным на валидной выборке. При этом переобучение для нашей модели присутствует, но не критично.

Дополнительно мы проверили модель на адекватность и наша модель показала хорошие результаты в сравнении со случайными предсказаниями.

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x] Jupyter Notebook открыт
- [ ] Весь код исполняется без ошибок
- [ ] Ячейки с кодом расположены в порядке исполнения
- [ ] Выполнено задание 1: данные загружены и изучены
- [ ] Выполнено задание 2: данные разбиты на три выборки
- [ ] Выполнено задание 3: проведено исследование моделей
    - [ ] Рассмотрено больше одной модели
    - [ ] Рассмотрено хотя бы 3 значения гипепараметров для какой-нибудь модели
    - [ ] Написаны выводы по результатам исследования
- [ ] Выполнено задание 3: Проведено тестирование
- [ ] Удалось достичь accuracy не меньше 0.75
